Anomaly Detection
Anomaly detection is the process of identifying unusual patterns or instances that deviate from the norm. In quality inspection, it plays a vital role in identifying defects that could impact product reliability, safety, or aesthetic appeal. Automated anomaly detection models can enhance quality control by rapidly and accurately flagging defects, which is essential in industries with high precision standards.

MVTec-AD Dataset
The MVTec-AD dataset is specifically designed for industrial anomaly detection and contains a variety of product images, each categorized as either normal (without defects) or anomalous (with specific defect types, such as cracks or contamination). It serves as a benchmark for evaluating the ability of models to distinguish between standard and defective products, aiding in the development of reliable quality inspection solutions.

Coresets (used in PatchCore)
Coresets are compact subsets of data that retain critical information while reducing redundancy. Used in PatchCore, they help optimize memory usage and processing speed by focusing on essential features, enabling faster and more efficient anomaly detection without sacrificing performance. This is particularly useful for large datasets or real-time applications where resource constraints are a factor.

In [1]:
pip install anomalib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade anomalib



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
!pip install qdrant-client


python(10212) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Image Preparation: First, we load an image from our files and get it ready for the model. This involves a few steps like resizing it to a specific size, turning it into a tensor (which is a format that models can understand), and normalizing the pixel values so that everything is on the same scale.


Device Management: Next, the code checks what kind of hardware is available for processing. It can either use Apple's Metal Performance Shaders (which is super efficient for graphics) or fall back to the CPU. This is important because we want the model to run as quickly as possible.

Anomaly Detection: After preparing the image, we feed it into the EfficientAd model. The model processes the image and produces something called an anomaly map. This map shows us where the model thinks there are unusual areas in the image—basically, spots that don’t fit what it has learned to be normal.


Visualization: Finally, we want to see the results! The code displays the original image, the anomaly map, and a combined version that overlays the two. This way, we can easily see where the model has detected anomalies in relation to the original image.

In [3]:
import numpy
print(numpy.__version__)

1.24.3


In [5]:
!pip install kornia
!pip install --upgrade kornia
!pip install einops
!pip install FrEIA
!pip install open_clip_torch



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import site
print(site.getsitepackages())

['/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages']


In [36]:
# ls '/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/anomalib/data/mvtec-ad'

ls: /Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/anomalib/data/mvtec-ad: No such file or directory


In [17]:
from anomalib.models import __all__
print(__all__)

['Cfa', 'Cflow', 'Csflow', 'Dfkde', 'Dfm', 'Draem', 'Dsr', 'EfficientAd', 'Fastflow', 'Fre', 'Ganomaly', 'Padim', 'Patchcore', 'ReverseDistillation', 'Rkde', 'Stfpm', 'Uflow', 'VlmAd', 'WinClip', 'AiVad']


In [18]:
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.models import Patchcore

# Specify your local path for the MVTec categories
mvt_dataset_path = "/Users/debikad/Desktop/AI/eng-ai-agents/datasets/imagenette/leather"

def train_and_evaluate_patchcore(categories):
    results = {}
    
    for category in categories:
        print(f"\nTraining and evaluating PatchCore on {category}")
        
        # Reinitialize the Engine for each category
        engine = Engine(max_epochs=5)  # Set the maximum number of epochs for training
        
        try:
            # Initialize the datamodule for the current category
            datamodule = MVTec(
                category=category,
                root=mvt_dataset_path,
                train_batch_size=1,
                num_workers=4
            )
            print(f"Data module for '{category}' initialized successfully.")
        except Exception as e:
            print(f"Error initializing data module for '{category}': {e}")
            continue
        
        # Initialize the PatchCore model
        model = Patchcore()
        
        # Fit the model to the data
        try:
            engine.fit(datamodule=datamodule, model=model)
            print(f"Model trained for '{category}'.")
        except Exception as e:
            print(f"Error during training for '{category}': {e}")
            continue
        
        # Test the model after training
        try:
            # Run the test and print tensor dimensions for debugging
            scores = engine.test(datamodule=datamodule, model=model)
            
            # Log the shape of scores for each category
            if hasattr(scores, 'shape'):
                print(f"Scores shape for '{category}': {scores.shape}")
            else:
                print(f"Scores for '{category}' do not have a 'shape' attribute.")
            
            print(f"Scores for '{category}': {scores}")
            results[category] = scores
        except IndexError as e:
            print(f"IndexError encountered during testing for '{category}': {e}")
        except Exception as e:
            print(f"Unexpected error during testing for '{category}': {e}")
    
    return results

# Specify the categories to train on
categories = ["tile", "grid", "leather"]

# Train and evaluate PatchCore
print("Starting PatchCore Training and Evaluation...")
patchcore_results = train_and_evaluate_patchcore(categories)
print("PatchCore Results:", patchcore_results)


Starting PatchCore Training and Evaluation...

Training and evaluating PatchCore on tile
Data module for 'tile' initialized successfully.


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_metrics | MetricCollection         | 0      | train
3 | image_threshold       | F1AdaptiveThreshold      | 0      

Training: |          | 0/? [00:00<?, ?it/s]

/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/ligh

Validation: |          | 0/? [00:00<?, ?it/s]

...

/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.
python(5506) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5507) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5508) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5509) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/ligh

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │     0.994535505771637     │
│        pixel_AUROC        │    0.9898852109909058     │
│       pixel_F1Score       │    0.44042396545410156    │
└───────────────────────────┴───────────────────────────┘

Scores for 'leather' do not have a 'shape' attribute.
Scores for 'leather': [{'pixel_AUROC': 0.9898852109909058, 'pixel_F1Score': 0.44042396545410156, 'image_AUROC': 1.0, 'image_F1Score': 0.994535505771637}]
PatchCore Results: {'tile': [{'pixel_AUROC': 0.9478980302810669, 'pixel_F1Score': 0.621834397315979, 'image_AUROC': 0.9866521954536438, 'image_F1Score': 0.9818181991577148}], 'grid': [{'pixel_AUROC': 0.9805421233177185, 'pixel_F1Score': 0.37902554869651794, 'image_AUROC': 0.9799498915672302, 'image_F1Score': 0.9649122953414917}], 'leather': [{'pixel_AUROC': 0.9898852109909058, 'pixel_F1Score': 0.44042396545410156, 'image_AUROC': 1.0, 'image_F1Score': 0.994535505771637}]}


In [21]:
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.models import EfficientAd

# Specify your local path for the MVTec categories
mvt_dataset_path = "/Users/debikad/Desktop/AI/eng-ai-agents/datasets/imagenette/leather"

def train_and_evaluate_efficientad(categories):
    results = {}
    
    for category in categories:
        print(f"\nTraining and evaluating EfficientAD on {category}")
        
        # Reinitialize the Engine for each category
        engine = Engine(max_epochs=5)  # Set the maximum number of epochs for training
        
        try:
            # Initialize the datamodule for the current category
            datamodule = MVTec(
                category=category,
                root=mvt_dataset_path,
                train_batch_size=1,
                num_workers=4
            )
            print(f"Data module for '{category}' initialized successfully.")
        except Exception as e:
            print(f"Error initializing data module for '{category}': {e}")
            continue
        
        # Initialize the EfficientAD model
        model = EfficientAd()  # This uses EfficientAD’s default settings
        
        # Fit the model to the data
        try:
            engine.fit(datamodule=datamodule, model=model)
            print(f"Model trained for '{category}'.")
        except Exception as e:
            print(f"Error during training for '{category}': {e}")
            continue
        
        # Test the model after training
        try:
            # Run the test and print tensor dimensions for debugging
            scores = engine.test(datamodule=datamodule, model=model)
            
            # Log the shape of scores for each category
            if hasattr(scores, 'shape'):
                print(f"Scores shape for '{category}': {scores.shape}")
            else:
                print(f"Scores for '{category}' do not have a 'shape' attribute.")
            
            print(f"Scores for '{category}': {scores}")
            results[category] = scores
        except IndexError as e:
            print(f"IndexError encountered during testing for '{category}': {e}")
        except Exception as e:
            print(f"Unexpected error during testing for '{category}': {e}")
    
    return results

# Specify the categories to train on
categories = ["tile", "grid", "leather"]

# Train and evaluate EfficientAD
print("Starting EfficientAD Training and Evaluation...")
efficientad_results = train_and_evaluate_efficientad(categories)
print("EfficientAD Results:", efficientad_results)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead


Starting EfficientAD Training and Evaluation...

Training and evaluating EfficientAD on tile
Data module for 'tile' initialized successfully.



  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | EfficientAdModel         | 8.1 M  | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_metrics | MetricCollection         | 0      | train
3 | image_threshold       | F1AdaptiveThreshold      | 0      | train
4 | pixel_threshold       | F1AdaptiveThreshold      | 0      | train
5 | image_metrics         | AnomalibMetricCollection | 0      | train
6 | pixel_metrics         | AnomalibMetricCollection | 0      | train
---------------------------------------------------------------------------
8.1 M     Trainable params
0         Non-trainable params
8.1 M     Total params
32.235    Total estimated model params size (MB)
46        Modules in train mode
7         Modules in eval mode
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/lightning/pytorch/t

Training: |          | 0/? [00:00<?, ?it/s]

python(5633) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5634) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5635) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(5636) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/d

...

/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.
python(9332) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(9333) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(9334) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(9335) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/debikad/Desktop/AI/eng-ai-agents/.venv/lib/python3.11/site-packages/kornia/feature/ligh

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9222147464752197     │
│       image_F1Score       │    0.9058823585510254     │
│        pixel_AUROC        │    0.9555909633636475     │
│       pixel_F1Score       │    0.5339598059654236     │
└───────────────────────────┴───────────────────────────┘

Scores for 'leather' do not have a 'shape' attribute.
Scores for 'leather': [{'pixel_AUROC': 0.9555909633636475, 'pixel_F1Score': 0.5339598059654236, 'image_AUROC': 0.9222147464752197, 'image_F1Score': 0.9058823585510254}]
EfficientAD Results: {'tile': [{'pixel_AUROC': 0.8370459079742432, 'pixel_F1Score': 0.581669807434082, 'image_AUROC': 0.9790764451026917, 'image_F1Score': 0.9447852969169617}], 'grid': [{'pixel_AUROC': 0.9069438576698303, 'pixel_F1Score': 0.4609525799751282, 'image_AUROC': 0.9841270446777344, 'image_F1Score': 0.9821428656578064}], 'leather': [{'pixel_AUROC': 0.9555909633636475, 'pixel_F1Score': 0.5339598059654236, 'image_AUROC': 0.9222147464752197, 'image_F1Score': 0.9058823585510254}]}


In [33]:
from anomalib.models import EfficientAd, Patchcore

def load_model_for_category(model_class, category):
    checkpoint_path = f"/Users/debikad/Desktop/AI/eng-ai-agents/results/{model_class.__name__}/MVTec/{category}/latest/weights/lightning/model.ckpt"
    model = model_class.load_from_checkpoint(checkpoint_path)
    model.eval()  # Set the model to evaluation mode
    return model

# Load models for each category
categories = ["grid", "leather", "tile"]

# Create a dictionary to store the models
models = {
    "EfficientAD": {},
    "PatchCore": {}
}

# Load EfficientAD models
for category in categories:
    models["EfficientAD"][category] = load_model_for_category(EfficientAd, category)

# Load PatchCore models
for category in categories:
    models["PatchCore"][category] = load_model_for_category(Patchcore, category)

# Now you can access the models like this:
efficientad_grid_model = models["EfficientAD"]["grid"]
patchcore_grid_model = models["PatchCore"]["grid"]

efficientad_leather_model = models["EfficientAD"]["leather"]
patchcore_leather_model = models["PatchCore"]["leather"]

efficientad_tile_model = models["EfficientAD"]["tile"]
patchcore_tile_model = models["PatchCore"]["tile"]

# Example usage for extracting features and adding to Qdrant
# Add to Qdrant or perform other tasks as required


In [107]:
import matplotlib.pyplot as plt
from anomalib.data.utils import read_image
from torchvision import transforms
from PIL import Image
import torch
import numpy as np

# Set the image path
example_image_path = "/Users/debikad/Desktop/AI/eng-ai-agents/datasets/imagenette/leather/grid/train/good/000.png"

# Load the image
image = read_image(example_image_path)
if isinstance(image, np.ndarray):
    image = Image.fromarray((image * 255).astype(np.uint8))  # Convert numpy array to PIL image if needed

# Define the transformation pipeline
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Match model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Apply transformations to the image
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Determine the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Move the model and input tensor to the same device
efficientad_grid_model.to(device)  # Ensure the model is on the same device
image_tensor = image_tensor.to(device)  # Move the input tensor to the same device

# Generate prediction and obtain the anomaly map
with torch.no_grad():
    output_dict = efficientad_grid_model(image_tensor)  # Forward pass to get the anomaly map

# Extract the anomaly map from the output dictionary
anomaly_map = output_dict['anomaly_map']  # Use the key for the anomaly map

# Check the output shape of the anomaly map
print(f"Anomaly Map Shape: {anomaly_map.shape}")

# Convert the anomaly map to a numpy array for visualization
anomaly_map_np = anomaly_map.squeeze(0).cpu().numpy()  # Remove batch dimension

# Ensure the anomaly map is in the correct format for visualization
if anomaly_map_np.ndim == 3:
    anomaly_map_np = anomaly_map_np.squeeze(0)  # Remove channel dimension if present

# Plot the results
# Plot the results
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Display the original image
axs[0].imshow(image)
axs[0].axis('off')
axs[0].set_title("Original Image")

# Display the anomaly map
axs[1].imshow(anomaly_map_np, cmap='hot')
axs[1].axis('off')
axs[1].set_title("Anomaly Map")

# Overlay the anomaly map on the original image
axs[2].imshow(image, alpha=0.7)  # Show the original image with transparency
axs[2].imshow(anomaly_map_np, cmap='hot', alpha=0.3)  # Overlay heatmap with transparency
axs[2].axis('off')
axs[2].set_title("Overlayed Heatmap")

plt.tight_layout()

# Save the figure to a file
plt.savefig("anomaly_detection_results.png")  # Save as PNG file
plt.close(fig)  # Close the figure to free up memory


Anomaly Map Shape: torch.Size([1, 1, 256, 256])


In [35]:
from torchvision import transforms
from PIL import Image
import torch 
def extract_features(model, image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.Resize((256, 256)),  # Adjust size as necessary
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    image_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Forward pass through the model
    with torch.no_grad():
        features = model(image_tensor).cpu().numpy()

    return features.flatten()  # Return a 1D feature vector


In [36]:
# def add_to_qdrant(client, features, image_id):
#     client.upload_collection(
#         collection_name="anomaly_features",
#         vectors=[features],
#         payload=[{"image_id": image_id}]
#     )
# def find_similar_images(client, query_features):
#     search_result = client.search(
#         collection_name="anomaly_features",
#         query_vector=query_features,
#         limit=5
#     )
#     return [hit.payload["image_id"] for hit in search_result]


In [116]:
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from anomalib.models import Patchcore
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sklearn.decomposition import PCA
import os
import random
import uuid

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

client = QdrantClient(":memory:")

collection_name = 'anomaly_features'

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=50, distance=Distance.COSINE)
)

pca = PCA(n_components=50)

def extract_features(model, image_path):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    model.to(device)
    model.eval()
    with torch.no_grad():
        features = model(image)
        if isinstance(features, dict):
            features = features['anomaly_map']
    
    features = features.squeeze().cpu().numpy()
    
    if features.ndim == 1:
        features = features.reshape(1, -1)
    elif features.ndim == 3:
        features = features.reshape(features.shape[0], -1)
    
    if features.shape[0] > 1 and features.shape[1] > 50:
        reduced_features = pca.fit_transform(features)
    else:
        reduced_features = features
    
    return reduced_features.flatten()[:50]

def add_to_qdrant(image_id, feature, metadata):
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=feature.tolist(),
        payload=metadata
    )
    client.upsert(
        collection_name=collection_name,
        points=[point]
    )

def process_dataset(dataset_path, model):
    for split in ["train", "test"]:
        split_path = os.path.join(dataset_path, split)
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                for idx, image_name in enumerate(os.listdir(class_path)):
                    image_path = os.path.join(class_path, image_name)
                    image_id = f"{split}_{class_name}_{idx}"
                    
                    features = extract_features(model, image_path)
                    metadata = {
                        'original_id': image_id,
                        'image_path': image_path,
                        'split': split,
                        'class': class_name
                    }
                    add_to_qdrant(image_id=image_id, feature=features, metadata=metadata)
                    
                    print(f"Processed: {image_id}")

def find_similar_images(query_image_path, model, top_k=5):
    query_features = extract_features(model, query_image_path)
    
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_features.tolist(),
        limit=top_k
    )
    return search_result

if __name__ == "__main__":
    dataset_path = "/Users/debikad/Desktop/AI/eng-ai-agents/datasets/imagenette/leather/leather"

    # Load your PatchCore model
    patchcore_model = patchcore_leather_model

    try:
        process_dataset(dataset_path, patchcore_model)

        query_class = random.choice(["good"])
        query_image_path = os.path.join(dataset_path, "test", query_class, os.listdir(os.path.join(dataset_path, "test", query_class))[0])
        
        similar_images = find_similar_images(query_image_path, patchcore_model)

        print(f"Query image: {query_image_path}")
        print(f"Query model: {type(patchcore_model).__name__}")
        print("Similar images:")
        for result in similar_images:
            print(f"ID: {result.payload['original_id']}, Score: {result.score}, Path: {result.payload['image_path']}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

Using device: mps


/var/folders/bb/st8mqs7n6dl2vsyc4_39_ts00000gn/T/ipykernel_86065/4051202123.py:21: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Processed: train_good_0
Processed: train_good_1
Processed: train_good_2
Processed: train_good_3
Processed: train_good_4
Processed: train_good_5
Processed: train_good_6
Processed: train_good_7
Processed: train_good_8
Processed: train_good_9
Processed: train_good_10
Processed: train_good_11
Processed: train_good_12
Processed: train_good_13
Processed: train_good_14
Processed: train_good_15
Processed: train_good_16
Processed: train_good_17
Processed: train_good_18
Processed: train_good_19
Processed: train_good_20
Processed: train_good_21
Processed: train_good_22
Processed: train_good_23
Processed: train_good_24
Processed: train_good_25
Processed: train_good_26
Processed: train_good_27
Processed: train_good_28
Processed: train_good_29
Processed: train_good_30
Processed: train_good_31
Processed: train_good_32
Processed: train_good_33
Processed: train_good_34
Processed: train_good_35
Processed: train_good_36
Processed: train_good_37
Processed: train_good_38
Processed: train_good_39
Processed: